In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
os.chdir("/Users/bytedance/MLTrader-deploy/")
import sys
sys.path.insert(0, ".")
import pandas as pd
from research.orderbook_strategies.utils.factor_analysis import update_signal_results_db
from research.orderbook_strategies.utils.helper import load
import numpy as np
from scipy.stats import spearmanr
import research.orderbook_strategies.utils.helper as helper
import research.orderbook_strategies.utils.factor_analysis as factor_analysis

In [9]:
factor_analysis.update_signal_results_db('signal_result_with_atr')

update all signal results


In [12]:
period_list = [32, 64, 128, 256, 512, 1024, 2048]
dire_signal_list = ["kdj.j", "buy_power", "adx.direc2.1024", "rmi", "volume.advantage", 
                    "kdj.k", "ma.dif.10", "dbook", "range.pos", "nr", 
                    "dbook.spread.diff", "dbook.spread"]
dire_signal_list_period = [signal+".{}".format(period) for period in period_list for signal in dire_signal_list]
range_signal_list = ["std", "trend.index", "atr", "adx.1024"]

dire_signal_with_variance = np.array([])
dire_signal_with_variance_period = []
for dire_signal in dire_signal_list:
    for range_signal in range_signal_list:
        signal_name = dire_signal+"."+range_signal +".2048"
        dire_signal_with_variance = np.append(dire_signal_with_variance, signal_name)
dire_signal_with_variance = list(dire_signal_with_variance)
dire_signal_with_variance_period = [signal+".{}".format(period) for period in period_list for signal in dire_signal_with_variance]
trades_signal = ['trade.buy_power.more_position.12', 'trade.doublebuy.value.ratio', 'trade.buy_power', 'trade.buy_power_count', 'trade.buy_power']
trades_signal_period = [signal+".{}".format(period) for period in np.power(2, range(10, 15)) for signal in trades_signal]

print("#dire = {}, #dire_with_var = {}, #trades = {}".format(len(dire_signal_list_period), len(dire_signal_with_variance_period), len(trades_signal_period)))

#dire = 84, #dire_with_var = 336, #trades = 25


In [13]:
product_list = ["OPUSDT","BTCUSDT", "ETHUSDT", "APTUSDT", "1000PEPEUSDT"]
all_results = pd.read_csv("/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal.csv")

all_signals = all_results['signal_name'].unique()
for signal_name in dire_signal_list_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))
        
for signal_name in dire_signal_with_variance_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))
        
for signal_name in trades_signal_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))

trade.buy_power.more_position.12.1024 not exist
trade.doublebuy.value.ratio.1024 not exist
trade.buy_power.1024 not exist
trade.buy_power_count.1024 not exist
trade.buy_power.1024 not exist
trade.buy_power.more_position.12.2048 not exist
trade.doublebuy.value.ratio.2048 not exist
trade.buy_power.2048 not exist
trade.buy_power_count.2048 not exist
trade.buy_power.2048 not exist
trade.buy_power.more_position.12.4096 not exist
trade.doublebuy.value.ratio.4096 not exist
trade.buy_power.4096 not exist
trade.buy_power_count.4096 not exist
trade.buy_power.4096 not exist
trade.buy_power.more_position.12.8192 not exist
trade.doublebuy.value.ratio.8192 not exist
trade.buy_power.8192 not exist
trade.buy_power_count.8192 not exist
trade.buy_power.8192 not exist
trade.buy_power.more_position.12.16384 not exist
trade.doublebuy.value.ratio.16384 not exist
trade.buy_power.16384 not exist
trade.buy_power_count.16384 not exist
trade.buy_power.16384 not exist


In [14]:
grouper_names = ["grouper_0", "grouper_1", "grouper_2", "grouper_3"]
all_results["grouper_0"] = all_results["signal_name"].astype(str) + "_" + all_results["reverse"].astype(str)
all_results["grouper_1"] = all_results["grouper_0"] + "_" + all_results["product"].astype(str)
all_results["grouper_2"] = all_results["grouper_1"] + "_" + all_results["open"].astype(str)
all_results["grouper_3"] = all_results["grouper_2"] + "_" + all_results["close"].astype(str)

## 先filter 交易次数过少的因子
stats = all_results.pivot_table(index="grouper_0", columns="is_train", aggfunc="median", values='avg.num')
prev = len(stats.index)
valid_str = list(stats[stats[True] > 1].index)
now_have = len(valid_str)
all_results = all_results.query("grouper_0 in @valid_str")
print("keep {}/{}".format(now_have, prev))

keep 588/843


### Learderboard for Direc Signal Top20 (全Symbol中位数)

In [43]:
# group_0
sub_res = all_results[np.isin(all_results["signal_name"], dire_signal_list_period)]
group_0_res = sub_res.groupby(by=["grouper_0", "is_train"]).median()
group_0_res_train = group_0_res.loc[(slice(None), True), :].droplevel(1)
group_0_res_test = group_0_res.loc[(slice(None), False), :].droplevel(1)

cols_of_interest = ["sharpe.ret", "avg.num", "avg.ret"]
good_groups_in_train = list(group_0_res_train[group_0_res_train['sharpe.ret'] > 0].index)
print("#good_in_train: {}/{}".format(len(good_groups_in_train), group_0_res_train.shape[0]))

test_perf_of_good = group_0_res_test[np.isin(group_0_res_test.index, good_groups_in_train)][cols_of_interest]
test_perf_of_good["sharpe.ret.train"] = test_perf_of_good.index.map(group_0_res_train['sharpe.ret'])

# performance without transaction cost
test_perf_of_good.sort_values(by="sharpe.ret.train", ascending=False).head(20)

#good_in_train: 43/114


sharpe.ret    avg.num   avg.ret  sharpe.ret.train
grouper_0                                                                 
range.pos.64_-1            0.144861   7.073643  0.000012          1.452959
volume.advantage.32_-1     0.161001  19.315891  0.000011          1.295063
kdj.j.128_-1               0.263086   6.264210  0.000030          1.197517
kdj.k.128_-1              -0.124558   9.282946 -0.000006          1.017383
volume.advantage.64_-1     0.290576  11.732558  0.000022          0.825879
volume.advantage.256_-1    0.420705   4.183764  0.000073          0.787381
rmi.32_-1                 -0.213175  11.570847 -0.000017          0.737305
rmi.64_-1                  0.177408   6.888814  0.000022          0.684367
kdj.k.256_-1               0.305607   5.847436  0.000028          0.585335
range.pos.256_-1          -0.425943   2.668511 -0.000084          0.511898
kdj.k.512_-1              -0.155199   4.026018 -0.000026          0.504230
dbook.32_1                 1.587080  32.138168  0.000051          0.500896
volume.advantage.128_-1    0.205188   7.021199  0.000029          0.474208
range.pos.512_-1           0.480847   1.664103  0.000170          0.460390
dbook.spread.128_-1       -0.204692   1.513372 -0.000050          0.458423
nr.128_-1                  0.320401   7.608050  0.000053          0.457079
rmi.128_-1                -0.337674   3.969260 -0.000057          0.435252
volume.advantage.512_-1    0.502644   2.743232  0.000187          0.397552
range.pos.128_-1          -0.406658   4.563153 -0.000065          0.378563
volume.advantage.2048_1   -1.076867   1.213459 -0.000890          0.371985

### Learderboard for Direc Signal Top20 (分Symbol中位数)

In [44]:
# group_0
sub_res = all_results[np.isin(all_results["signal_name"], dire_signal_list_period)]
group_0_res = sub_res.groupby(by=["grouper_1", "is_train"]).median()
group_0_res_train = group_0_res.loc[(slice(None), True), :].droplevel(1)
group_0_res_test = group_0_res.loc[(slice(None), False), :].droplevel(1)

cols_of_interest = ["sharpe.ret", "avg.num", "avg.ret"]
good_groups_in_train = list(group_0_res_train[group_0_res_train['sharpe.ret'] > 0].index)
print("#good_in_train: {}/{}".format(len(good_groups_in_train), group_0_res_train.shape[0]))

test_perf_of_good = group_0_res_test[np.isin(group_0_res_test.index, good_groups_in_train)][cols_of_interest]
test_perf_of_good["sharpe.ret.train"] = test_perf_of_good.index.map(group_0_res_train['sharpe.ret'])
test_perf_of_good['product'] = pd.Series(test_perf_of_good.index).map(lambda x: x.split("_")[-1]).values
# performance without transaction cost
for product in product_list:
    display(test_perf_of_good[test_perf_of_good['product'] == product].sort_values(by="sharpe.ret.train", ascending=False).head(10))

#good_in_train: 212/570


sharpe.ret    avg.num   avg.ret  sharpe.ret.train  \
grouper_1                                                                    
kdj.k.512_-1_OPUSDT        0.173561   5.521318  0.000030          1.537544   
range.pos.64_-1_OPUSDT    -0.836934  10.422481 -0.000062          1.255256   
kdj.k.1024_-1_OPUSDT       0.624984   2.798450  0.000160          1.237924   
range.pos.256_-1_OPUSDT    0.040463   3.705426  0.000007          1.226860   
kdj.j.512_-1_OPUSDT        0.472893   3.321705  0.000118          1.184801   
kdj.j.128_-1_OPUSDT       -0.506398   9.608527 -0.000038          1.129159   
range.pos.512_-1_OPUSDT    0.473693   2.023256  0.000121          1.066582   
range.pos.128_-1_OPUSDT   -1.503431   6.127907 -0.000467          0.947739   
nr.2048_-1_OPUSDT          1.233376   2.488372  0.000998          0.756447   
kdj.k.256_-1_OPUSDT       -1.226698   9.000000 -0.000217          0.723812   

                        product  
grouper_1                        
kdj.k.512_-1_OPUSDT      OPUSDT  
range.pos.64_-1_OPUSDT   OPUSDT  
kdj.k.1024_-1_OPUSDT     OPUSDT  
range.pos.256_-1_OPUSDT  OPUSDT  
kdj.j.512_-1_OPUSDT      OPUSDT  
kdj.j.128_-1_OPUSDT      OPUSDT  
range.pos.512_-1_OPUSDT  OPUSDT  
range.pos.128_-1_OPUSDT  OPUSDT  
nr.2048_-1_OPUSDT        OPUSDT  
kdj.k.256_-1_OPUSDT      OPUSDT

sharpe.ret    avg.num   avg.ret  \
grouper_1                                                         
rmi.32_-1_BTCUSDT                 0.944657   0.615385  0.000132   
dbook.spread.diff.32_1_BTCUSDT    1.901988  51.292308  0.000020   
range.pos.64_-1_BTCUSDT           0.687009   0.138462  0.000179   
dbook.32_1_BTCUSDT                2.680085   4.926923  0.000128   
rmi.64_-1_BTCUSDT                 0.510273   1.053846  0.000176   
volume.advantage.32_-1_BTCUSDT    3.098785   3.373077  0.000130   
kdj.j.64_-1_BTCUSDT               1.738645   0.138462  0.000169   
kdj.k.128_-1_BTCUSDT              0.135146   0.069231  0.000037   
dbook.spread.128_-1_BTCUSDT      -0.527665   0.034615 -0.000110   
ma.dif.10.64_-1_BTCUSDT           1.795420   3.692308  0.000119   

                                sharpe.ret.train  product  
grouper_1                                                  
rmi.32_-1_BTCUSDT                       2.462280  BTCUSDT  
dbook.spread.diff.32_1_BTCUSDT          2.329285  BTCUSDT  
range.pos.64_-1_BTCUSDT                 1.757135  BTCUSDT  
dbook.32_1_BTCUSDT                      1.735396  BTCUSDT  
rmi.64_-1_BTCUSDT                       1.733843  BTCUSDT  
volume.advantage.32_-1_BTCUSDT          1.485010  BTCUSDT  
kdj.j.64_-1_BTCUSDT                     1.373965  BTCUSDT  
kdj.k.128_-1_BTCUSDT                    1.362593  BTCUSDT  
dbook.spread.128_-1_BTCUSDT             1.358282  BTCUSDT  
ma.dif.10.64_-1_BTCUSDT                 1.219238  BTCUSDT

sharpe.ret   avg.num   avg.ret  \
grouper_1                                                        
kdj.j.128_-1_ETHUSDT              1.540277  0.192308  0.000129   
ma.dif.10.32_-1_ETHUSDT           1.883586  5.153846  0.000080   
range.pos.64_-1_ETHUSDT           0.916314  0.492308  0.000133   
kdj.k.256_-1_ETHUSDT              1.371437  0.430769  0.000116   
nr.128_-1_ETHUSDT                 0.653119  0.419231  0.000095   
dbook.32_1_ETHUSDT                3.127482  8.057692  0.000091   
volume.advantage.32_-1_ETHUSDT    0.561334  2.496154  0.000028   
kdj.k.128_-1_ETHUSDT              3.276297  0.265385  0.000450   
range.pos.32_-1_ETHUSDT           3.651169  0.753846  0.000348   
volume.advantage.64_-1_ETHUSDT    0.000000  1.653846  0.000000   

                                sharpe.ret.train  product  
grouper_1                                                  
kdj.j.128_-1_ETHUSDT                    2.466401  ETHUSDT  
ma.dif.10.32_-1_ETHUSDT                 2.432456  ETHUSDT  
range.pos.64_-1_ETHUSDT                 2.319376  ETHUSDT  
kdj.k.256_-1_ETHUSDT                    2.142311  ETHUSDT  
nr.128_-1_ETHUSDT                       2.139976  ETHUSDT  
dbook.32_1_ETHUSDT                      2.109414  ETHUSDT  
volume.advantage.32_-1_ETHUSDT          2.080575  ETHUSDT  
kdj.k.128_-1_ETHUSDT                    2.070209  ETHUSDT  
range.pos.32_-1_ETHUSDT                 1.978111  ETHUSDT  
volume.advantage.64_-1_ETHUSDT          1.965708  ETHUSDT

sharpe.ret    avg.num   avg.ret  \
grouper_1                                                          
rmi.256_1_APTUSDT                 -1.082750   2.545802 -0.000245   
adx.direc2.1024.512_1_APTUSDT      0.000000   3.702290  0.000000   
adx.direc2.1024.256_1_APTUSDT     -0.031388   4.396947 -0.000007   
kdj.j.512_1_APTUSDT               -1.277332   2.912214 -0.000231   
volume.advantage.1024_1_APTUSDT   -1.008771   1.927481 -0.000670   
volume.advantage.32_-1_APTUSDT     0.099791  21.045802  0.000007   
kdj.k.1024_1_APTUSDT               0.614063   2.656489  0.000315   
rmi.128_1_APTUSDT                 -0.164621   4.717557 -0.000077   
rmi.512_1_APTUSDT                 -0.398768   1.438931 -0.000164   
volume.advantage.512_1_APTUSDT    -1.060288   3.041985 -0.000368   

                                 sharpe.ret.train  product  
grouper_1                                                   
rmi.256_1_APTUSDT                        1.539504  APTUSDT  
adx.direc2.1024.512_1_APTUSDT            1.335014  APTUSDT  
adx.direc2.1024.256_1_APTUSDT            1.301371  APTUSDT  
kdj.j.512_1_APTUSDT                      1.234922  APTUSDT  
volume.advantage.1024_1_APTUSDT          1.192847  APTUSDT  
volume.advantage.32_-1_APTUSDT           1.128692  APTUSDT  
kdj.k.1024_1_APTUSDT                     1.054474  APTUSDT  
rmi.128_1_APTUSDT                        1.014603  APTUSDT  
rmi.512_1_APTUSDT                        0.904576  APTUSDT  
volume.advantage.512_1_APTUSDT           0.884545  APTUSDT

sharpe.ret    avg.num   avg.ret  \
grouper_1                                                                
range.pos.512_1_1000PEPEUSDT            -0.858689   2.408333 -0.000516   
kdj.k.512_1_1000PEPEUSDT                 0.028898   5.866667  0.000005   
dbook.spread.32_1_1000PEPEUSDT           1.228112   4.366667  0.000253   
nr.64_-1_1000PEPEUSDT                   -0.404082  23.700000 -0.000049   
dbook.32_1_1000PEPEUSDT                  1.787961  57.050000  0.000061   
nr.128_-1_1000PEPEUSDT                   0.180498  16.008333  0.000023   
range.pos.256_1_1000PEPEUSDT             0.828051   3.691667  0.000177   
kdj.j.256_1_1000PEPEUSDT                -2.208344   6.166667 -0.000258   
volume.advantage.256_-1_1000PEPEUSDT    -0.021765   7.475000 -0.000007   
dbook.spread.diff.1024_1_1000PEPEUSDT    0.053973   6.883333  0.000014   

                                       sharpe.ret.train       product  
grouper_1                                                              
range.pos.512_1_1000PEPEUSDT                   2.484670  1000PEPEUSDT  
kdj.k.512_1_1000PEPEUSDT                       2.454581  1000PEPEUSDT  
dbook.spread.32_1_1000PEPEUSDT                 2.358948  1000PEPEUSDT  
nr.64_-1_1000PEPEUSDT                          2.332312  1000PEPEUSDT  
dbook.32_1_1000PEPEUSDT                        2.277777  1000PEPEUSDT  
nr.128_-1_1000PEPEUSDT                         2.205314  1000PEPEUSDT  
range.pos.256_1_1000PEPEUSDT                   2.163583  1000PEPEUSDT  
kdj.j.256_1_1000PEPEUSDT                       2.059400  1000PEPEUSDT  
volume.advantage.256_-1_1000PEPEUSDT           2.047828  1000PEPEUSDT  
dbook.spread.diff.1024_1_1000PEPEUSDT          2.003713  1000PEPEUSDT

### Learderboard for Interaction Signal Top20 (全Symbol中位数)

# (Direc Signal)构造Portfolio后的Test Return Learderboard

In [76]:
results = factor_analysis.get_signal_performance_result(dire_signal_list_period, "signal_result_with_atr", product_list, 1e-4, 1)

In [77]:
trend_res = results['trend.signal.stat']
reve_res = results["reverse.signal.stat"]
signal_trend_res = trend_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']
signal_rever_res = reve_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']

trend_signals = signal_trend_res[(signal_trend_res - signal_rever_res) > 0].index
reverse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) < 0].index
nouse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) == 0].index

print("#trend={}, #reverse={}, #nouse={}".format(len(trend_signals), len(reverse_signals), len(nouse_signals)))

merge_res = pd.concat([trend_res.loc[np.isin(trend_res['signal'], trend_signals)], 
                       reve_res.loc[np.isin(reve_res['signal'], reverse_signals)]], 
                      axis=0)

display(merge_res.sort_values(by='trainSharpe', ascending=False))

#trend=38, #reverse=36, #nouse=10


signal  reverse  num_good_product  trainSharpe  testSharpe  \
5                kdj.k.32       -1                 2     4.229345    2.789503   
24              kdj.j.128       -1                 4     3.666122    1.707982   
17               kdj.k.64       -1                 3     3.631148    1.868213   
20           range.pos.64       -1                 5     3.563911    2.062242   
29              kdj.k.128       -1                 5     3.556452    1.641319   
..                    ...      ...               ...          ...         ...   
69                nr.1024        1                 2     1.067901   -2.159308   
76  volume.advantage.2048       -1                 4     1.037994    0.896804   
80         range.pos.2048        1                 5     0.979423   -0.132015   
78         ma.dif.10.2048        1                 4     0.802338   -0.496753   
75               rmi.2048        1                 5     0.615771   -0.574665   

    trainAvgDailyTradeNum  testAvgDailyTradeNum  sum_good_strat  avgTrainRet  \
5               50.783587             38.246154             NaN     0.003965   
24              58.797775             39.661869             NaN     0.003684   
17              76.679746             49.007543             NaN     0.004547   
20             135.552122             95.312854             NaN     0.004581   
29             145.497059            103.410684             NaN     0.004715   
..                    ...                   ...             ...          ...   
69               5.665777              7.341869            25.0     0.003248   
76              13.546973             11.816711             NaN     0.003123   
80              17.897810             15.878798            53.0     0.002920   
78              14.659084              7.713772            43.0     0.004240   
75              12.847467             10.644511            47.0     0.002038   

    avgTestRet  
5     0.001580  
24    0.000861  
17   -0.000224  
20    0.000574  
29    0.000332  
..         ...  
69   -0.006460  
76    0.000970  
80    0.000139  
78   -0.000413  
75   -0.000471  

[74 rows x 10 columns]

# # (Direc With Var Signal)构造Portfolio后的Test Return Learderboard

In [78]:
results = factor_analysis.get_signal_performance_result(dire_signal_with_variance_period, "signal_result_with_atr", product_list, 1e-4, 1)

open file failed: /Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.32.reverse.pkl, e=[Errno 2] No such file or directory: '/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.32.reverse.pkl'
/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.32.reverse.pkl not found 
open file failed: /Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.64.trend.pkl, e=[Errno 2] No such file or directory: '/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.64.trend.pkl'
/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.64.tre

In [79]:
trend_res = results['trend.signal.stat']
reve_res = results["reverse.signal.stat"]
signal_trend_res = trend_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']
signal_rever_res = reve_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']

trend_signals = signal_trend_res[(signal_trend_res - signal_rever_res) > 0].index
reverse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) < 0].index
nouse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) == 0].index

print("#trend={}, #reverse={}, #nouse={}".format(len(trend_signals), len(reverse_signals), len(nouse_signals)))

merge_res = pd.concat([trend_res.loc[np.isin(trend_res['signal'], trend_signals)], 
                       reve_res.loc[np.isin(reve_res['signal'], reverse_signals)]], 
                      axis=0)

display(merge_res.sort_values(by='trainSharpe', ascending=False))

#trend=153, #reverse=155, #nouse=28


signal  reverse  num_good_product  \
20                    kdj.k.std.2048.32       -1                 1   
91   dbook.spread.diff.adx.1024.2048.64        1                 1   
43   dbook.spread.diff.adx.1024.2048.32        1                 1   
155   adx.direc2.1024.adx.1024.2048.256       -1                 1   
99              kdj.j.adx.1024.2048.128       -1                 5   
..                                  ...      ...               ...   
325            nr.trend.index.2048.2048       -1                 5   
302                   rmi.atr.2048.2048        1                 4   
202        adx.direc2.1024.atr.2048.512        1                 5   
277            nr.trend.index.2048.1024       -1                 5   
250       adx.direc2.1024.atr.2048.1024        1                 5   

     trainSharpe  testSharpe  trainAvgDailyTradeNum  testAvgDailyTradeNum  \
20      5.522623   -1.860022               9.368421              9.548246   
91      5.025148    0.121712              69.400000             28.467262   
43      4.954294   -0.377528              57.644898             24.960317   
155     4.750805    2.603924               5.745238              5.600000   
99      3.815531   -0.254969              77.128873             57.193799   
..           ...         ...                    ...                   ...   
325     0.711335    1.572744              19.622001             19.064463   
302     0.700247   -0.809005              14.567847              8.108694   
202     0.679841   -0.779695              23.866640             10.322095   
277     0.664394    1.100236              25.901751             24.541063   
250     0.599677   -1.377065              18.133310              8.842326   

     sum_good_strat  avgTrainRet  avgTestRet  
20              NaN     0.004960   -0.000775  
91             31.0     0.021904    0.000245  
43             23.0     0.017055   -0.000973  
155             NaN     0.009389    0.008191  
99              NaN     0.003774   -0.001422  
..              ...          ...         ...  
325             NaN     0.002161    0.002819  
302            48.0     0.003025   -0.001288  
202            53.0     0.002899   -0.001319  
277             NaN     0.001839    0.001913  
250            61.0     0.002378   -0.002573  

[308 rows x 10 columns]